In [1]:
# 2020-12-25 we will prepare for top and bottom 10% of RNA 500kb bins

In [1]:
import pandas as pd
import numpy as np

In [2]:
from math import ceil

In [3]:
def get_top_bot(RNA, time, percent = 0.1):
    '''we will get top and bot file out'''
    data = pd.read_csv('strict.' + RNA + '.' + time + '.500kb.relaxed', sep = '\t')
    total = data.shape[0]
    target = ceil(total * percent)
    top = data.iloc[:target, :]
    bot = data.iloc[-target:, :]
    top.to_csv('top10.' + RNA + '.' + time + '.500kb', sep = '\t', index = False)
    bot.to_csv('bot10.' + RNA + '.' + time + '.500kb', sep = '\t', index = False)

In [4]:
for i in ['Fn1', 'Macf1', 'Neat1', 'Runx1']:
    get_top_bot(i, '60h')
get_top_bot('Hmga2', '0h')

In [70]:
# 2020-12-29 after we get all the result, (we use raw result), we need to remove non-exist trans interaction (9.28e-13), then we
# get the same number for boxplot

In [103]:
def prepare_for_look(RNA):
    '''we will get file for look'''
    # for trans
    neat1 = pd.read_csv('BOX.top_vs_bot.'+RNA+'.raw', sep = '\t')
    top = neat1.loc[(neat1['RNA_type']=='trans') & (neat1['rank']=='top')].copy()
    #top.sort_values('interaction', ascending = False, inplace = True)
    new_top = top.loc[top['interaction'] > 1e-12]
    bot = neat1.loc[(neat1['RNA_type']=='trans') & (neat1['rank']=='bot')].copy()
    bot.sort_values('interaction', ascending = False, inplace = True)
    new_bot = bot.loc[bot['interaction'] > 1e-12]
    
    count = new_bot.shape[0]
    
    final_top = new_top.sample(n=count, random_state = 20201229).copy()
    final_top.sort_values('interaction', ascending = False, inplace = True)
    
    # for cis
    top_cis = neat1.loc[(neat1['RNA_type']=='cis') & (neat1['rank']=='top')].copy()
    #top.sort_values('interaction', ascending = False, inplace = True)
    new_top_cis = top_cis.loc[top_cis['interaction'] > 1e-12]
    bot_cis = neat1.loc[(neat1['RNA_type']=='cis') & (neat1['rank']=='bot')].copy()
    bot_cis.sort_values('interaction', ascending = False, inplace = True)
    new_bot_cis = bot_cis.loc[bot_cis['interaction'] > 1e-12]
    
    count_cis = new_bot_cis.shape[0]
    
    final_top_cis = new_top_cis.sample(n=count_cis, random_state = 20201229).copy()
    final_top_cis.sort_values('interaction', ascending = False, inplace = True)
    
    
    output = pd.concat([final_top, new_bot, final_top_cis, new_bot_cis], axis = 0)
    return output

In [104]:
a = prepare_for_look('Neat1')

In [105]:
a.shape[0]

2614

In [106]:
a

,RNA_type,interaction,rank
434,trans,4.437408,top
453,trans,3.806109,top
437,trans,3.630904,top
4452,trans,3.404463,top
2816,trans,3.343203,top
...,...,...,...
20400,cis,1.216518,bot
20403,cis,1.172361,bot
20290,cis,1.072382,bot
20426,cis,1.047655,bot


In [107]:
a.to_csv('BOX.top_vs_bot.Neat1.raw.rm_none', sep = '\t', index = False)

In [108]:
# I need a new fully-auto version

In [19]:
def prepare_for_final_boxplot(RNA):
    '''for cis and trans separately, we will control the same count for boxplot;
    in the mean time, we will remove non-exist hic interactions'''
    data = pd.read_csv('BOX.top_vs_bot.' + RNA + '.raw', sep = '\t')
    rm_none = data.loc[data['interaction'] > 1e-12].copy()
    # for trans
    top_trans = rm_none.loc[(rm_none['RNA_type'].str.startswith('trans')) & (rm_none['rank']=='top')].copy()
    bot_trans = rm_none.loc[(rm_none['RNA_type'].str.startswith('trans')) & (rm_none['rank']=='bot')].copy()
    count_trans = min([top_trans.shape[0], bot_trans.shape[0]])
    new_top_trans = top_trans.sample(n = count_trans, random_state = 20201229).copy()
    new_bot_trans = bot_trans.sample(n = count_trans, random_state = 20201229).copy()
    # for cis
    top_cis = rm_none.loc[(rm_none['RNA_type'].str.startswith('cis')) & (rm_none['rank']=='top')].copy()
    bot_cis = rm_none.loc[(rm_none['RNA_type'].str.startswith('cis')) & (rm_none['rank']=='bot')].copy()
    count_cis = min([top_cis.shape[0], bot_cis.shape[0]])
    new_top_cis = top_cis.sample(n = count_cis, random_state = 20201229).copy()
    new_bot_cis = bot_cis.sample(n = count_cis, random_state = 20201229).copy()
    
    output = pd.concat([new_top_trans, new_bot_trans, new_top_cis, new_bot_cis], axis = 0)
    
    return output

In [7]:
a = prepare_for_final_boxplot('Fn1')

In [8]:
a.shape[0]

526

In [9]:
a.to_csv('BOX.top_vs_bot.Fn1.raw.rm_none', sep = '\t', index = False)

In [20]:
for i in ['Fn1', 'Runx1', 'Macf1', 'Macf1', 'Hmga2', 'Neat1']:
    tmp = prepare_for_final_boxplot(i)
    tmp.to_csv('BOX.top_vs_bot.' + i + '.raw.rm_none', sep = '\t', index = False, header = False)

In [21]:
# 2021-1-7, we will mege regions of each RNA and check for ICF and DLR

In [34]:
def get_data(RNA, top_bot, time):
    '''we will get data from each RNA'''
    data = pd.read_csv(top_bot + '.' + RNA + '.' + time + '.500kb', sep = '\t', names = ['id', 'median_' + RNA], header = 0)
    data.set_index('id', inplace = True)
    return data

In [41]:
all_RNA = []
for i in ['Macf1', 'Runx1', 'Neat1']:
    tmp = get_data(i, 'top10', '60h')
    all_RNA.append(tmp)

In [42]:
new = pd.concat(all_RNA, axis = 1, join = 'inner')

In [43]:
new

,median_Macf1,median_Runx1,median_Neat1
id,,,
chr17:39500000-40000000,2.680976,2.157308,2.706493
chr15:102000000-102500000,1.347988,0.966259,0.975351
chr15:61500000-62000000,1.058059,1.070704,0.888262
chr15:59000000-59500000,0.995611,0.920667,0.876396
chr14:77500000-78000000,0.929884,0.910690,0.897693
chr11:78500000-79000000,0.888118,1.270305,1.270747


In [38]:
all_RNA

[                           median_Fn1
 id                                   
 chr17:39500000-40000000      5.034295
 chr11:96500000-97000000      1.764005
 chr4:88000000-88500000       1.755818
 chr15:97500000-98000000      1.595389
 chr11:75500000-76000000      1.573724
 chr2:84000000-84500000       1.551288
 chr11:74500000-75000000      1.493451
 chr2:25000000-25500000       1.478009
 chr11:61500000-62000000      1.468327
 chr16:15000000-15500000      1.458668
 chr13:99000000-99500000      1.442301
 chr11:71500000-72000000      1.431777
 chr11:70000000-70500000      1.431083
 chr15:53000000-53500000      1.384589
 chr11:64000000-64500000      1.379416
 chr11:119000000-119500000    1.355602
 chr11:101000000-101500000    1.345291
 chr11:96000000-96500000      1.319704
 chr12:65000000-65500000      1.317394
 chr11:77500000-78000000      1.298041
 chr15:80500000-81000000      1.297380
 chr2:60000000-60500000       1.292919
 chr15:97000000-97500000      1.252663
 chr17:34000000-34500000 

In [45]:
# 2021-1-11 we will compare 0h and 60h

In [178]:
def process_each_caRNA(RNA):
    '''we will process each caRNA'''
    name_list = ['id_1', 'id_2', 'RNA_0h_1', 'RNA_0h_2', 'RNA_60h_1', 'RNA_60h_2', 'DNA_0h', 'DNA_60h', 'type']
    data = pd.read_csv('top10.' + RNA + '.60h.500kb.combinations.raw.full_time', names = name_list, sep = '\t')
    data['RNA_0h'] = data['RNA_0h_1'] + data['RNA_0h_2']
    data['RNA_60h'] = data['RNA_60h_1'] + data['RNA_60h_2']
    data['RNA_change'] = data['RNA_60h'] - data['RNA_0h']
    data['DNA_change'] = data['DNA_60h'] - data['DNA_0h']
    #data['DNA_log2FC'] = np.log2(data['DNA_60h'] / data['DNA_0h'])
    new = data.loc[:, ['id_1', 'id_2', 'RNA_0h', 'RNA_60h', 'DNA_0h', 'DNA_60h', 'RNA_change', 'DNA_change', 'type']].copy()
    rm_none = new.loc[(new['DNA_60h'] > 1e-12) & (new['DNA_change'] > 0)].copy()
    rm_none.sort_values(['id_1', 'id_2'], ascending = False, inplace = True)
    rm_none['id'] = rm_none['id_1'] + '_' + rm_none['id_2']
    tmp = rm_none.loc[:, ['id', 'RNA_0h', 'RNA_60h', 'DNA_0h', 'DNA_60h', 'RNA_change', 'DNA_change', 'type']]
    tmp.sort_values('DNA_60h', ascending = False, inplace = True)
    return tmp

In [179]:
fn1 = process_each_caRNA('Fn1')
neat1 = process_each_caRNA('Neat1')
macf1 = process_each_caRNA('Macf1')
runx1 = process_each_caRNA('Runx1')

In [180]:
fn1.head(n = 10)

,id,RNA_0h,RNA_60h,DNA_0h,DNA_60h,RNA_change,DNA_change,type
232,chr11:75500000-76000000_chr11:116000000-116500000,1.118591,2.728092,12.542258,13.583701,1.609501,1.041443,cis
593,chr15:53000000-53500000_chr15:97000000-97500000,0.000000,2.637252,11.612249,11.637061,2.637252,0.024812,cis
1195,chr15:74500000-75000000_chr15:92500000-93000000,0.000000,2.327077,8.717866,8.792375,2.327077,0.074508,cis
323,chr11:74500000-75000000_chr11:116000000-116500000,1.118591,2.647819,8.033604,8.325450,1.529228,0.291846,cis
300,chr11:74500000-75000000_chr11:101000000-101500000,0.000000,2.838742,8.131789,8.196817,2.838742,0.065028,cis
548,chr11:70000000-70500000_chr11:119000000-119500000,0.772027,2.786685,7.577130,7.703102,2.014658,0.125972,cis
386,chr11:61500000-62000000_chr11:119000000-119500000,0.772027,2.823928,6.397310,7.522897,2.051901,1.125587,cis
301,chr11:74500000-75000000_chr11:96000000-96500000,0.000000,2.813155,7.052101,7.368654,2.813155,0.316554,cis
1127,chr8:57500000-58000000_chr8:109500000-110000000,0.000000,2.349628,5.166760,6.898967,2.349628,1.732207,cis
415,chr11:61500000-62000000_chr11:120000000-120500000,0.000000,2.614656,6.775748,6.832607,2.614656,0.056858,cis


In [181]:
neat1

,id,RNA_0h,RNA_60h,DNA_0h,DNA_60h,RNA_change,DNA_change,type
4115,chr15:50500000-51000000_chr15:51000000-51500000,0.746877,2.105930,5.953226e+02,625.637049,1.359053,30.314443,cis
35721,chr1:183500000-184000000_chr1:184000000-184500000,0.847977,1.826317,5.074374e+02,517.376924,0.978340,9.939533,cis
18956,chr1:74500000-75000000_chr1:75000000-75500000,0.831612,1.971548,2.644708e+02,267.837236,1.139936,3.366402,cis
14224,chr8:7000000-7500000_chr8:8000000-8500000,0.000000,1.949865,2.469071e+02,255.529162,1.949865,8.622036,cis
25362,chr13:15500000-16000000_chr13:16000000-16500000,1.055320,1.897340,2.259169e+02,233.636398,0.842020,7.719484,cis
...,...,...,...,...,...,...,...,...
19263,chr2:156500000-157000000_chr4:63000000-63500000,0.000000,1.922527,1.000000e-12,0.149137,1.922527,0.149137,trans
46195,chr10:98000000-98500000_chr14:70000000-70500000,0.264190,1.742403,1.000000e-12,0.148020,1.478213,0.148020,trans
46177,chr1:157000000-157500000_chr14:70000000-70500000,0.264190,1.742887,1.000000e-12,0.138348,1.478697,0.138348,trans
45118,chr1:90000000-90500000_chr7:83500000-84000000,1.219632,1.755350,1.000000e-12,0.134935,0.535719,0.134935,trans


In [160]:
runx1.head(n = 10)

,id,RNA_0h,RNA_60h,DNA_0h,DNA_60h,RNA_change,DNA_change,type
9524,chr11:7000000-7500000_chr11:95500000-96000000,0.000000,2.056304,1.000000e-12,1.420264,2.056304,1.420264,cis
12729,chr10:32000000-32500000_chr10:127000000-127500000,0.000000,1.899565,1.000000e-12,1.042125,1.899565,1.042125,cis
7945,chr1:79000000-79500000_chr1:194500000-195000000,0.000000,2.030964,1.000000e-12,1.028318,2.030964,1.028318,cis
13830,chr15:100500000-101000000_chr8:126000000-12650...,1.059342,1.877431,1.000000e-12,0.996138,0.818089,0.996138,trans
12505,chr1:182500000-183000000_chr10:94000000-94500000,0.000000,1.990638,1.000000e-12,0.821168,1.990638,0.821168,trans
2852,chr11:117000000-117500000_chr8:125500000-12600...,0.000000,2.364109,1.000000e-12,0.809184,2.364109,0.809184,trans
8333,chr11:35000000-35500000_chr11:120500000-121000000,0.000000,2.008667,1.000000e-12,0.773150,2.008667,0.773150,cis
12359,chr11:4000000-4500000_chr2:127500000-128000000,0.000000,1.932054,1.000000e-12,0.771747,1.932054,0.771747,trans
10204,chr15:33500000-34000000_chr6:145000000-145500000,0.000000,2.050234,1.000000e-12,0.771353,2.050234,0.771353,trans
15128,chr18:62000000-62500000_chr2:127500000-128000000,0.000000,1.842356,1.000000e-12,0.762872,1.842356,0.762872,trans


In [161]:
macf1.head(n = 10)

,id,RNA_0h,RNA_60h,DNA_0h,DNA_60h,RNA_change,DNA_change,type
1363,chr15:81500000-82000000_chr17:7000000-7500000,0.0,2.584434,1.000000e-12,1.298598,2.584434,1.298598,trans
7486,chr12:111500000-112000000_chr8:124500000-12500...,0.0,2.154282,1.000000e-12,0.872192,2.154282,0.872192,trans
8515,chr17:7000000-7500000_chr9:64000000-64500000,0.0,2.047472,1.000000e-12,0.767836,2.047472,0.767836,trans
4333,chr13:99000000-99500000_chr2:26500000-27000000,0.0,2.241473,1.000000e-12,0.703330,2.241473,0.703330,trans
2433,chr15:97000000-97500000_chr18:62000000-62500000,0.0,2.488923,1.000000e-12,0.649848,2.488923,0.649848,trans
8051,chr18:62000000-62500000_chr2:128000000-128500000,0.0,2.122876,1.000000e-12,0.648299,2.122876,0.648299,trans
5255,chr15:76500000-77000000_chr2:37500000-38000000,0.0,2.378520,1.000000e-12,0.631571,2.378520,0.631571,trans
6960,chr1:181500000-182000000_chr18:75000000-75500000,0.0,2.194314,1.000000e-12,0.627156,2.194314,0.627156,trans
6841,chr11:54500000-55000000_chr9:77500000-78000000,0.0,2.289053,1.000000e-12,0.598688,2.289053,0.598688,trans
1873,chr2:76500000-77000000_chr15:76500000-77000000,0.0,2.668302,1.000000e-12,0.590690,2.668302,0.590690,trans


In [162]:
def merge_any_two_RNA(RNA_1, RNA_2, rna_1, rna_2):
    '''we will merge two RNA files'''
    merged = pd.merge(RNA_1, RNA_2, how = 'inner', on = 'id', suffixes = ['_' + rna_1, '_' + rna_2])
    merged.sort_values(['DNA_change_' + rna_1, 'DNA_change_' + rna_2], ascending = False, inplace = True)
    tmp = merged.loc[:, ['id', 'RNA_change_' + rna_1, 'RNA_change_' + rna_2, 'DNA_change_' + rna_1, 'DNA_change_' + rna_2]]
    return tmp

In [163]:
a = merge_any_two_RNA(neat1, macf1, 'neat1', 'macf1')

In [164]:
a

,id,RNA_change_neat1,RNA_change_macf1,DNA_change_neat1,DNA_change_macf1
0,chr15:12000000-12500000_chr17:28000000-28500000,0.536329,2.560582,0.567909,0.567909
1,chr11:120000000-120500000_chr15:61500000-62000000,-0.066285,2.174538,0.185777,0.185777


In [165]:
b = merge_any_two_RNA(neat1, runx1, 'neat1', 'runx1')

In [166]:
b

,id,RNA_change_neat1,RNA_change_runx1,DNA_change_neat1,DNA_change_runx1
0,chr11:117000000-117500000_chr8:125500000-12600...,1.403621,2.364109,0.809184,0.809184
1,chr11:105500000-106000000_chr15:53000000-53500000,0.472603,2.020126,0.442981,0.442981
2,chr11:96500000-97000000_chr15:62500000-63000000,0.595511,1.874355,0.250462,0.250462


In [167]:
c = merge_any_two_RNA(runx1, macf1, 'runx1', 'macf1')

In [168]:
c

,id,RNA_change_runx1,RNA_change_macf1,DNA_change_runx1,DNA_change_macf1
0,chr15:97000000-97500000_chr18:62000000-62500000,1.931181,2.488923,0.649848,0.649848


In [132]:
# 2021-1-11 for RNA DNA correlation

In [153]:
def process_each_caRNA(RNA):
    '''we will process each caRNA'''
    name_list = ['id_1', 'id_2', 'RNA_0h_1', 'RNA_0h_2', 'RNA_60h_1', 'RNA_60h_2', 'DNA_0h', 'DNA_60h', 'type']
    data = pd.read_csv('top10.' + RNA + '.60h.500kb.combinations.raw.full_time', names = name_list, sep = '\t')
    data['RNA_0h'] = data['RNA_0h_1'] + data['RNA_0h_2']
    data['RNA_60h'] = data['RNA_60h_1'] + data['RNA_60h_2']
    
    #data['DNA_log2FC'] = np.log2(data['DNA_60h'] / data['DNA_0h'])
    new = data.loc[:, ['id_1', 'id_2', 'RNA_0h', 'RNA_60h', 'DNA_0h', 'DNA_60h', 'type']].copy()
    rm_none = new.loc[((new['DNA_0h'] > 1e-12) & (new['DNA_60h'] > 1e-12)) & (new['type'] == 'trans')].copy()
    
    c = rm_none['DNA_0h']
    d = rm_none['DNA_60h']
    
    c[c<=1e-12]=np.nan
    c.fillna(0.0001,inplace=True)
    
    d[d<=1e-12]=np.nan
    d.fillna(0.0001,inplace=True)
    
    rm_none['RNA_change'] = rm_none['RNA_60h'] - rm_none['RNA_0h']
    rm_none['DNA_change'] = rm_none['DNA_60h'] - rm_none['DNA_0h']
    
    rm_none.sort_values(['id_1', 'id_2'], ascending = False, inplace = True)
    rm_none['id'] = rm_none['id_1'] + '_' + rm_none['id_2']
    tmp = rm_none.loc[:, ['id', 'RNA_0h', 'RNA_60h', 'DNA_0h', 'DNA_60h', 'RNA_change', 'DNA_change', 'type']]
    tmp.sort_values('DNA_change', ascending = False, inplace = True)
    return tmp

In [154]:
for i in ['Fn1', 'Macf1', 'Neat1', 'Runx1']:
    tmp = process_each_caRNA(i)
    tmp.to_csv('TIME_BOXPLOT.' + i, sep = '\t', index = False)

/home/ljj/00.soft/anaconda3/envs/coolbox/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/ljj/00.soft/anaconda3/envs/coolbox/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ljj/00.soft/anaconda3/envs/coolbox/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [138]:
a = process_each_caRNA('Fn1')

/home/ljj/00.soft/anaconda3/envs/coolbox/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/ljj/00.soft/anaconda3/envs/coolbox/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [139]:
a

,id,RNA_0h,RNA_60h,DNA_0h,DNA_60h,RNA_change,DNA_change,type
1127,chr8:57500000-58000000_chr8:109500000-110000000,0.000000,2.349628,5.166760,6.898967,2.349628,1.732207,cis
612,chr15:53000000-53500000_chr15:92500000-93000000,0.000000,2.537256,2.654708,4.230179,2.537256,1.575471,cis
386,chr11:61500000-62000000_chr11:119000000-119500000,0.772027,2.823928,6.397310,7.522897,2.051901,1.125587,cis
232,chr11:75500000-76000000_chr11:116000000-116500000,1.118591,2.728092,12.542258,13.583701,1.609501,1.041443,cis
805,chr11:77500000-78000000_chr8:125000000-125500000,0.000000,2.515473,0.000100,0.780425,2.515473,0.780325,trans
...,...,...,...,...,...,...,...,...
691,chr11:119000000-119500000_chr11:120500000-1210...,0.772027,2.494875,73.741596,51.713984,1.722849,-22.027612,cis
1303,chr11:120000000-120500000_chr11:121000000-1215...,0.000000,2.267379,93.235324,70.354488,2.267379,-22.880836,cis
1300,chr11:120000000-120500000_chr11:120500000-1210...,0.000000,2.285603,261.520382,227.596216,2.285603,-33.924167,cis
688,chr11:119000000-119500000_chr11:120000000-1205...,0.772027,2.501931,163.646787,117.193213,1.729904,-46.453573,cis
